In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import modin.pandas as pd
from fastai.tabular import *
import time
from datetime import datetime

2019-05-14 20:40:59,680	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-14_20-40-59_4103/logs.
2019-05-14 20:40:59,784	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:54832 to respond...
2019-05-14 20:40:59,890	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:12652 to respond...
2019-05-14 20:40:59,892	INFO services.py:804 -- Starting Redis shard with 10.0 GB max memory.
2019-05-14 20:40:59,898	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-14_20-40-59_4103/logs.
2019-05-14 20:40:59,900	WARNING services.py:1304 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 33736691712 bytes available. This may slow down performance! You may be able to free up space by deleting files in /dev/shm or terminating any running plasma_store_server processes. If you are inside a Docker container, you may need to pass an argument with the flag '--shm-

## 数据清洗

In [3]:
root = Path('../test_A')
root
root.ls()

PosixPath('../test_A')

[PosixPath('../test_A/ad_static_feature.out'),
 PosixPath('../test_A/ad_operation.dat'),
 PosixPath('../test_A/.ipynb_checkpoints'),
 PosixPath('../test_A/imps_log'),
 PosixPath('../test_A/test_sample.dat'),
 PosixPath('../test_A/user')]

In [4]:
u_data = root/'user'
u_data.ls()
log_data = root/'imps_log'
log_data.ls()

[PosixPath('../test_A/user/user_data')]

[PosixPath('../test_A/imps_log/totalExposureLog.out')]

- 清洗掉 ad_static_feature 中没有的广告
- 我们只关心有效广告的曝光情况

In [6]:
ad_static = pd.read_csv('../data/ad_static.csv', low_memory=False, encoding='utf-8')
ad_static.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸
0,593323,2015-04-07 09:43:55,26889,-1,3,85,64
1,146563,2015-06-04 09:11:32,23511,-1,3,118,64
2,216969,2016-03-10 06:35:34,26970,-1,1,33,1
3,357401,2016-03-10 06:35:35,26970,-1,1,33,1
4,542734,2016-03-10 06:35:35,26970,-1,1,33,34


In [7]:
ad_static.shape

(497666, 7)

### 广告操作数据

In [8]:
col_names = ['广告id', '创建/修改时间', '操作类型', '修改字段', '操作后的字段值']

In [9]:
ad_operation = pd.read_csv(root/'ad_operation.dat', sep='\t', header=None, names=col_names)

In [10]:
ad_operation.describe()

,广告id,创建/修改时间,操作类型,修改字段
count,760866.000000,7.608660e+05,760866.000000,760866.000000
mean,366502.509025,1.717885e+13,1.147152,1.523331
std,210931.328143,7.192564e+12,0.354258,0.855417
min,31.000000,0.000000e+00,1.000000,1.000000
25%,184651.000000,2.019022e+13,1.000000,1.000000
50%,363843.000000,2.019030e+13,1.000000,1.000000
75%,547318.000000,2.019031e+13,1.000000,2.000000
max,736053.000000,2.019032e+13,2.000000,4.000000


In [11]:
ad_operation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760866 entries, 0 to 760865
Data columns (total 5 columns):
广告id       760866 non-null int64
创建/修改时间    760866 non-null int64
操作类型       760866 non-null int64
修改字段       760866 non-null int64
操作后的字段值    760866 non-null object
dtypes: int64(4), object(1)
memory usage: 29.0+ MB


In [12]:
ad_operation.head(7)

,广告id,创建/修改时间,操作类型,修改字段,操作后的字段值
0,593323,0,2,2,90
1,593323,0,2,3,all
2,593323,0,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,20190217000000,1,1,0
4,593323,20190218233855,1,2,90
5,593323,20190218000000,1,1,1
6,593323,20190230000000,1,1,0


In [13]:
ad_operation.shape

(760866, 5)

In [14]:
ad_operation.操作类型.unique()

array([2, 1])

In [15]:
a = datetime.strptime('20190218233855', "%Y%m%d%H%M%S")

In [16]:
str(a)

'2019-02-18 23:38:55'

In [17]:
pd.to_datetime(a)

Timestamp('2019-02-18 23:38:55')

- 非法时间

In [18]:
def isVaildDate(date_str):
    try:
        time.strptime(date_str, "%Y%m%d%H%M%S")
        return True
    except:
        return False

In [19]:
def invalid_date(df, field):
    ret, l = [], len(df)
    df.reset_index(drop=True, inplace=True)  # 为了正常访问，重建索引
    for i in range(l):
        if df.loc[i, field] == 0:
            continue
        s = str(df.loc[i, field])
        if not isVaildDate(s):
            ret.append(i)
        
    # 删除行
    new_df = df.drop(ret, axis=0)
    return new_df

In [20]:
ad_operation = invalid_date(ad_operation, '创建/修改时间')

In [21]:
ad_operation.shape

(759574, 5)

In [22]:
ad_operation.head(7)

,广告id,创建/修改时间,操作类型,修改字段,操作后的字段值
0,593323,0,2,2,90
1,593323,0,2,3,all
2,593323,0,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,20190217000000,1,1,0
4,593323,20190218233855,1,2,90
5,593323,20190218000000,1,1,1
7,593323,20190301000000,1,1,1


 - 合并表格

In [23]:
ads = pd.merge(ad_static, ad_operation, on='广告id', how='inner')

In [24]:
ads.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
0,593323,2015-04-07 09:43:55,26889,-1,3,85,64,0,2,2,90
1,593323,2015-04-07 09:43:55,26889,-1,3,85,64,0,2,3,all
2,593323,2015-04-07 09:43:55,26889,-1,3,85,64,0,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,2015-04-07 09:43:55,26889,-1,3,85,64,20190217000000,1,1,0
4,593323,2015-04-07 09:43:55,26889,-1,3,85,64,20190218233855,1,2,90


In [25]:
ads.shape

(613409, 11)

- 非法时间
- 我们避免直接使用 for 循环

In [26]:
def date_mod(df_row):
    if df_row['创建/修改时间'] == 0:
        df_row['创建/修改时间'] = df_row['创建时间']
    else:
        df_row['创建/修改时间'] = datetime.strptime(str(df_row['创建/修改时间']), "%Y%m%d%H%M%S")
    return df_row

In [27]:
ads = ads.apply(date_mod, axis=1)

In [28]:
ads.tail()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,创建/修改时间,操作类型,修改字段,操作后的字段值
613404,209095,2019-03-19 13:21:51,25817,31956,18,198,64,2019-03-19 13:21:51,2,4,"281474976710655,281474976710655,28147497671065..."
613405,209095,2019-03-19 13:21:51,25817,31956,18,198,64,2019-03-19 21:23:09,1,2,20
613406,719982,2019-03-19 13:24:10,25817,31956,18,232,54,2019-03-19 13:24:10,2,2,20
613407,719982,2019-03-19 13:24:10,25817,31956,18,232,54,2019-03-19 13:24:10,2,3,"behavior:22855,23131"
613408,719982,2019-03-19 13:24:10,25817,31956,18,232,54,2019-03-19 13:24:10,2,4,"281474976710655,281474976710655,28147497671065..."


In [29]:
ads.shape

(613409, 11)

In [30]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 613409 entries, 0 to 613408
Data columns (total 11 columns):
广告id       613409 non-null int64
创建时间       613409 non-null object
广告账户id     613409 non-null int64
商品id       613409 non-null int64
商品类型       613409 non-null int64
广告行业id     613409 non-null int64
素材尺寸       613409 non-null object
创建/修改时间    613409 non-null object
操作类型       613409 non-null int64
修改字段       613409 non-null int64
操作后的字段值    613409 non-null object
dtypes: int64(7), object(4)
memory usage: 56.2+ MB


- 数据保存

In [31]:
ads.to_csv('../data/ad_static_operation.csv', index=None, encoding='utf-8')